# SAS単体でUUID(バージョン3,5)を生成する処理

SASではUUIDを生成する関数としてUUIDGENがありますが、UUIDバージョン5には対応していない、ということでSAS単体で生成するコードを作成しました。  
動作についてはSAS Ondemandのutf-8環境で確認しています。おそらくshift-jisでも動くとは思いますが、うまく動作しないような場合はお知らせください。  
hashing関数を使用しているため、動作は**SAS 9.4M6以降**が対象になります。  
SAS 9.4M5以前で動かしたいというような場合、他の方が書かれたものですが、[こちらの記事](https://japelin.hatenablog.com/entry/2020/08/29/000824)などを参考に、hashing関数の部分を、ファイルに対象としたい文字列を書き込んでファイルのハッシュ値を取得するというような処理に差し替えれば可能とは思います。  



以下では出力の確認、およびUUIDの仕様について説明を行います。  

## 作成した関数の仕様
`uuid(5, "DNS","sas.com")`のような形で、バージョン、名前空間、文字列を指定します。  
基本的には最後の文字列以外は例の通りでいいかと思います。  

UUIDバージョン5を使うのがほとんどと思いますが、一応バージョン３にも対応できるようにしました。  
名前空間は`DNS`等か直接指定することが可能です。  
パラメータに想定されない値が指定された場合は、エラーメッセージが返り値となります。  

形としては関数的なマクロも考えていましたが、バイト列を扱う都合上扱いづらく、fcmpプロシージャを使う形としています。  
関数名、パラメータ等の調整や、処理を取り出してデータステップで行うなど、好みにより修正して利用してください。 

In [2]:
%include "uuid.sas" ;

data _null_ ;
  file print ;
  uuid = uuid(5, "DNS","sas.com") ;
  put uuid ;
run ;

## Python コードとの比較と確認

UUIDモジュールによる結果を確認しておきます。

上記のSASのコードを書く前に[uuid.py](https://github.com/python/cpython/blob/3.10/Lib/uuid.py)はバージョン5以外に関するコードやユーティリティ的なコードも含まれるので必要最低限なコードを整理して、再現ができているか確認しています。

In [3]:
%%python

# UUIDモジュールを使ってUUID5を生成
import uuid
print(uuid.uuid5(uuid.NAMESPACE_DNS, 'sas.com'))


# UUIDモジュールから処理を抽出して再現
from hashlib import sha1
def uuid5(namespace, name, version=5):
    namespace_bytes=int(namespace.strip('{}').replace('-', ''), 16).to_bytes(16, byteorder='big')
    hash_ = sha1(namespace_bytes + bytes(name, 'utf-8')).digest()

    int_ = int.from_bytes(hash_[:16], byteorder='big')
    int_ &= ~(0xc000 << 48)
    int_ |= 0x8000 << 48
    int_ &= ~(0xf000 << 64)
    int_ |= version << 76

    hex_ = '%032x' % int_
    return '%s-%s-%s-%s-%s' % (
            hex_[:8], hex_[8:12], hex_[12:16], hex_[16:20], hex_[20:])

            
print("再現による結果")
print(uuid5(namespace = '6ba7b810-9dad-11d1-80b4-00c04fd430c8', name = 'sas.com'))

28104995-0a79-5524-89ee-7ceeb8e5b2db
再現による結果
28104995-0a79-5524-89ee-7ceeb8e5b2db


## UUID仕様について

細かい仕様などは[UUID(wikipedia)](https://ja.wikipedia.org/wiki/UUID)等を見てもらえばいいと思うので、特にバージョン5を中心に要点を記載します。

- UUID概要  
128ビットの数値ですが、16進法による550e8400-e29b-41d4-a716-446655440000のような形で表されることが一般的です。

- バリアント  
`xxxxxxxx-xxxx-xxxx-Nxxx-xxxxxxxxxxxx` の Nの部分は特定の値になります。
Nの部分を2進表記にすると`10__`とするため、16進のNでは8,9,A,Bのいずれかになります。

- バージョン情報  
`xxxxxxxx-xxxx-Vxxx-Nxxx-xxxxxxxxxxxx` のVの部分はバージョン番号になります。
SASのUUIDGEN関数のバージョンは仕様に記載されていませんが、出力例`8343fd83-e635-438d-9277-550a5b8460f3`からバージョン4であることがわかります。

- バージョン5の生成方法  
  1. 名前空間(DNS, URL等)と名前をつなげてsha1でハッシュ値を計算
  2. 得られたハッシュ値(160ビット)から128ビットより後を切り捨て
  3. バリアントとバージョン情報を設定

このため、128ビットのうち122ビットはハッシュ値に由来し、残りの6ビットは固定の値になります。


## SASでの対応方法
可能であればPythonのUUIDライブラリのロジックをなるべくそのまま実装する形にしたかったのですが、UUIDが128ビットに対しSASのBLSHIFT関数などビット演算関数は仕様として32ビット分までという制約により、ビット演算の部分は2進表記の文字列に変換し、上記の仕様により直接値を指定することで対応することとしました。